#Задание 1.
Дообучите языковую модель на датасете инструкций, используя LoRA. Проверьте, что дообученная модель отличается от изначальной - сгенерируйте продолжения для одних и тех же промптов и сравните результаты.

Вы можете взять за основу код семинара PEFT, изменив датасет цитат на датасет инструкций (можно просто скопировать из семинара про General_instruct_fine-tuning). Можно использовать alpaca_dataset, датасет Dolly 2 или ваш собственный переведенный датасет (или все вместе). Важно использовать модель с большим количеством параметров (относительно семинара по General instruct fine-tuning). Например, можно попробовать OPT-1.3b,OPT-2.7b или ai-forever/rugpt3large_based_on_gpt2.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git@main
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 752.0 kB/s eta 0:00:00


Попробуем загрузить большую модель без дополнительных параметров.

In [2]:
from transformers import AutoModelForCausalLM

In [3]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-1.3b',
                                             cache_dir='./models').to('cuda')

Ошибки памяти, кстати, не произошло. Любопытно.

## FP16

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [8]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-1.3b',
                                             torch_dtype=torch.float16, # указываем fp16
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [10]:
batch = tokenizer("The house stood on a slight rise just on the edge of the village.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, no_repeat_ngram_size=2)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 The house stood on a slight rise just on the edge of the village.
The view from the front porch was of a small pond, surrounded by a low wall of trees. The house was built in the early 1800's, and was a simple, but elegant, structure. It was the home of John and Mary,


### Fine-tuning

In [13]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [14]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [18]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b",
    load_in_8bit=False,
    device_map='auto',
)

In [19]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [20]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

In [21]:
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [22]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [23]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6291456 || all params: 1322049536 || trainable%: 0.47588655558516074


### Обучение
Попробуем взять датасет Alpaca

In [36]:
import transformers


In [37]:
!wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

--2023-06-20 00:04:53--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘alpaca_data.json’

alpaca_data.json    100%[===================>]  21.72M  --.-KB/s    in 0.1s    

2023-06-20 00:04:55 (153 MB/s) - ‘alpaca_data.json’ saved [22773992/22773992]



In [41]:
import json

In [49]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer


In [42]:
data = json.load(open('alpaca_data.json'))

In [47]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [50]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [51]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [52]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [57]:
model_name = "facebook/opt-1.3b"

In [58]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=96,
    padding_side="right",
    use_fast=False,
)

In [59]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path="alpaca_data.json")
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [70]:
train_args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    )

In [71]:
trainer = transformers.Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

In [72]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,2.222100
2,2.302200
3,2.135800
4,2.188200
5,2.215300
6,2.188000
7,2.870000
8,2.396800
9,2.189200
10,2.056000


TrainOutput(global_step=400, training_loss=1.7031019991636276, metrics={'train_runtime': 490.8886, 'train_samples_per_second': 13.038, 'train_steps_per_second': 0.815, 'total_flos': 4446891379752960.0, 'train_loss': 1.7031019991636276, 'epoch': 0.12})

In [73]:
model.save_pretrained('opt_1.3_lora')

Чтобы загрузить обученную модель нужно сначала загрузить базовую модель, а потом применить к ней LoRa веса

In [1]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "opt_1.3_lora"

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-1.3b",
                                             return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [2]:
def generate(text, tokenizer, model):
  batch = tokenizer(text, return_tensors='pt').to('cuda')
  output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.0, no_repeat_ngram_size=2)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

##Предсказания для оценки базовой модели

In [11]:
generate("I have a dream that",  tokenizer, model)

"I have a dream that one day, the world will be a better place.\nI've got a feeling that's not going to happen."

In [12]:
generate("I want to know why",  tokenizer, model)

"I want to know why the fuck they are doing this.\nBecause they're a bunch of fucking idiots."

Неэтично.

In [13]:
generate("I want to know about",  tokenizer, model)

'I want to know about the "I\'m a girl" part.\nI think it\'s a joke about how the guy is a guy and the girl is just a dude.'

In [14]:
generate("I wish i was",  tokenizer, model)

"I wish i was a girl\nI'm a guy and I wish I was gay.\nYou're a gay guy?\nNo, I'm just a straight guy who wishes he was bi."

Gender issues. Интересно, вызвано ли это спецификой датасета.

In [17]:
generate("You know you're extremely tired when",  tokenizer, model)

'You know you\'re extremely tired when you can\'t even remember what you were doing last night.\nI was in the middle of a conversation with my friend and I was like "I\'m so tired, I can barely remember anything I did last week."'

Same, bro

In [16]:
generate("I am reading a book about",  tokenizer, model)

'I am reading a book about the history of the United States.\nI have been reading it for a few weeks now. I am about halfway through it. It is a very interesting book. The author, David McCullough, is an historian. He has written a lot of'

#Загрузим LoRA веса и попробуем те же промпты

In [18]:
model = PeftModel.from_pretrained(model, peft_model_id)


In [19]:
generate("I have a dream that",  tokenizer, model)

'I have a dream that one day, the world will be a better place.\nI will make it so.'

In [20]:
generate("I want to know why",  tokenizer, model)

"I want to know why the hell they are using a picture of a guy with a beard.\nBecause it's a man."

Генерируется более нейтральный, не агрессивный, текст.

In [21]:
generate("I want to know about",  tokenizer, model)

'I want to know about the "I\'m a girl" thing. I\'m not a guy, but I\'ve always wondered if it\'s a thing for guys.\nI\'ve never heard of it being a "thing" for men.  I think it is more of'

In [22]:
generate("I wish i was",  tokenizer, model)

"I wish i was a girl\nI'm a guy and I wish I was gay."

In [23]:
generate("You know you're extremely tired when",  tokenizer, model)

"You know you're extremely tired when you can't even remember what you were doing last night.\nI was watching a movie and I was so tired I couldn't remember the ending."

In [24]:
generate("I am reading a book about",  tokenizer, model)

'I am reading a book about the history of the United States and the Civil War. It is a fascinating read and I am learning a lot about this time period. I have been reading about how the war was fought and what the outcome was. The book is called "The Civil'

Тут есть ощущение, что название книжки сгенерировано не целиком. Но угадать можно

#Результат
Кажется, что после обучения модель работает лучше. По крайней мере, не выдаёт оскорблений. Тем не менее, заметно, что в двух примерах текст не дописан, а обрывается на полуслове.